In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/task1.train.txt', sep="\t", header=None, \
                        names=["text", "article_id", "label"])

In [ ]:
df.head()

,text,article_id,label
0,"Et tu, Rhody? A recent editorial in the Provi...",727600136,non-propaganda
1,A recent post in The Farmington Mirror — our t...,731714618,non-propaganda
2,"President Donald Trump, as he often does while...",731714635,non-propaganda
3,"February is Black History Month, and nothing l...",728627182,non-propaganda
4,"The snow was so heavy, whipped up by gusting w...",728627443,non-propaganda


In [ ]:
df.isnull().sum()

text          0
article_id    0
label         0
dtype: int64

In [ ]:
import plotly.graph_objects as go

labels = ["Propaganda", "Non-Propaganda"]
class_freq_count = [df[df["label"] == "propaganda"]["label"].count(), df[df["label"] == "non-propaganda"]["label"].count()]

fig = go.Figure([go.Bar(x=labels, y=class_freq_count, marker_color='orange')])

fig.update_layout(title_text='Articles Per Class')

fig.show()


In [ ]:
from glob import iglob
import pandas as pd
import re
import numpy as np

data = pd.DataFrame(columns=["article_id", "article", "propaganda"])
technique = dict()

for path in iglob("/content/drive/MyDrive/train-20240203T184513Z-001/train/*"):
    file_name = path.split("/")[-1]
    extensions = file_name.split(".")
    article_id = re.search("article([0-9]+)", extensions[0]).group(1)
    if article_id not in data.article_id.values:
        data = data.append({"article_id" : article_id , "article" : '', "propaganda" : ''}, ignore_index=True)
    if extensions[1] == "txt":
        with open(path, 'r') as f:
            data.loc[data.article_id == article_id, 'article'] = f.read()
    elif extensions[1] == "task2":
        with open(path, 'r') as f:
            data.loc[data.article_id == article_id, 'propaganda'] = f.read()
    elif extensions[1] == "task3":
        with open(path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                tech = line.split("\t")[1]
                if tech in technique:
                    technique[tech] += 1
                else:
                    technique[tech] = 1

<ipython-input-8-b2785c167097>:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-8-b2785c167097>:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-8-b2785c167097>:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-8-b2785c167097>:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-8-b2785c167097>:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-8-b2785c167097>:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pand

In [ ]:
data

,article_id,article,propaganda
0,999001188,"Dear Illegal Alien ‘Caravan’: Say, ‘Hola’ To T...",999001188\t1\tpropaganda\n999001188\t2\tnon-pr...
1,776368676,Trump To Sessions In Series Of Tweets: ‘Stop T...,776368676\t1\tpropaganda\n776368676\t2\tnon-pr...
2,999001621,This Guardian Fake News Story Proves That The ...,999001621\t1\tpropaganda\n999001621\t2\tnon-pr...
3,790667730,Senator Lindsey Graham Unleashes Firestorm At ...,790667730\t1\tpropaganda\n790667730\t2\tnon-pr...
4,790483991,"Viganò Doubles Down, Accuses Francis of Losing...",790483991\t1\tnon-propaganda\n790483991\t2\tno...
...,...,...,...
288,111111113,Kate Steinle's death at the hands of a Mexican...,111111113\t1\tnon-propaganda\n111111113\t2\tno...
289,729549972,NASA captures farthest ever image from Earth\n...,729549972\t1\tpropaganda\n729549972\t2\tnon-pr...
290,111111131,Trump's North Korean gamble ends with 'special...,111111131\t1\tnon-propaganda\n111111131\t2\tno...
291,697472447,Security Guard Jesus Campos Shot During The Ve...,697472447\t1\tnon-propaganda\n697472447\t2\tno...


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier,LogisticRegression
from sklearn import metrics
import nltk

In [ ]:

from nltk.corpus import stopwords

from nltk.stem import PorterStemmer
import tensorflow as tf

In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras import layers
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [ ]:
df.head()

,text,article_id,label
0,"Et tu, Rhody? A recent editorial in the Provi...",727600136,non-propaganda
1,A recent post in The Farmington Mirror — our t...,731714618,non-propaganda
2,"President Donald Trump, as he often does while...",731714635,non-propaganda
3,"February is Black History Month, and nothing l...",728627182,non-propaganda
4,"The snow was so heavy, whipped up by gusting w...",728627443,non-propaganda


In [ ]:
y = df['label']
X_train,X_test,y_train,y_test = train_test_split(df['text'],y,test_size =0.1, stratify=y)

In [ ]:
ps = PorterStemmer()
nltk.download('stopwords')
def my_tokenizer(str_input):
  str_input = str_input.replace("!"," exclamationsign")
  str_input = str_input.replace("?"," questionmark")
  str_input = str_input.replace("'"," singlequote")
  tokens = str_input.split()
  words = [ps.stem(token) for token in tokens if token not in stopwords.words('english')]
  return words


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english',
                                   tokenizer=my_tokenizer,
                                   lowercase=True,
                                   min_df=3,
                                   max_df=0.8,
                                   max_features=None,
                                   strip_accents='unicode',
                                   analyzer='word',
                                   token_pattern=r'\w{1,}',
                                   ngram_range=(1, 3),
                                   use_idf=True,
                                   smooth_idf=True,
                                   sublinear_tf=True)


In [17]:
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_train = tfidf_vectorizer.transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'thu', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi

In [18]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics

In [19]:
linear_clf = PassiveAggressiveClassifier(max_iter=200)
linear_clf.fit(tfidf_train, y_train)
pred = linear_clf.predict(tfidf_test)

In [22]:
score_pa_tfidf = metrics.accuracy_score(y_test, pred)
print("Accuracy:", score_pa_tfidf)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_pa = (2*P*R)/(P+R)
print("F1 score : ",F1_pa)

Accuracy: 0.9736037788274521
F1 score :  0.8700410396716826
